## Bell Tests for Quantum and Classical Variables

In [64]:
import random, time, math
shots = 1024

### Bell test for classical variables

In this notebook you will investigate how quantum variables (based on qubits) differ from standard ones (based on bits).

We'll do this by creating a pair of variables, which we will call `A` and `B`. These could be represented by any kind of data structure, and could be initialized by any kind of process. This could be deterministic or random. It could initialize the two variables in a correlated way, or they could be completely independent. Whatever you choose do do, do it in the function below.

As an example, I define `A` and `B` to be random floating point number, with `A` in the range $[0.0, 2/3)$ and `B` in the range $[0.0, 1/3)$. They are also correlated, since they are rescalings of the same result from `random.random()`.

In [65]:
def setup_variables ():
    
    ### Replace this section with anything you want ###
    
    r = random.random()
    
    A = r*(2/3)
    B = r*(1/3)
    
    ### End of section ###
    
    return A, B

We then define a hashing function that can be applied to either variable to produce a bit. So when given either one of the variables as an input, this function must return a bit value as output.

The hashing function must also be capable of performing two different types of hash. The hash type must therefore also be supplied as an input to the function.

To be consistent with the rest of the program, the two possible hash types should be called `'H'` and `'V'`. Also, the output must be in the form of a single value bit string: either `'0'` or `'1'`.

As an example, I created bits by comparing `A` and `B` to a certain value. The output is `'1'` if they are under that value, and `'0'` otherwise. The value used is 0.5 for an `'H'` type hash, and 0.25 for a `'V'` type hash.

In [30]:
def hash2bit ( variable, hash ):
    
    ### Replace this section with anything you want ###
    
    if hash=='V':
        bit = (variable<0.5)
    elif hash=='H':
        bit = (variable<0.25)
        
    bit = str(int(bit))
    
    ### End of section ###
        
    return bit

Once these are defined, there are four quantities we wish to calculate: `E['HH']`, `E['HV']`, `E['VH']` and `E['VV']`.

Let is focus on `E['HV']` as an example. This compares the bit value derived from an `'H'` type hash on `A` with that from a `'V'` type has on `B`. It is defined such that `E['HV']=1` if their values are always the same, and `E['HV']=-1` if they are always different.

If `A` and `B` are produced by a deterministic process, this means that `E['HV']` is simply

    hash2bit(A,'H')==hash2bit(B,'H') - hash2bit(A,'V')!=hash2bit(B,'V')
    
For a random process, however, we wish to sample many times and average over all the values found for this quantity.

This is done in the following function, which returns all the values of E in a dictionary.

In [31]:
def calculate_P ( ):
    
    P = {}
    for hashes in ['HH','HV','VH','VV']:
        
        # calculate each P[hashes] by sampling over `shots` samples
        P[hashes] = 0
        for shot in range(shots):

            A, B = setup_variables()

            a = hash2bit ( A, hashes[0] ) # hash type for variable `A` is the first chacter of `hashes`
            b = hash2bit ( B, hashes[1] ) # hash type for variable `B` is the second chacter of `hashes`

            P[hashes] += (a==b) / shots
 
    return P

In [83]:
def bell_test (P):
    
    for hashA in ['H','V']:
        for hashM in ['H','V']:
            for hashB in ['H','V']:

                hashes1 = hashA+hashM
                hashes2 = hashM+hashB
                hashesC = hashA+hashB

                p = P[hashesC] 
                lower_bound = P[hashes1] * P[hashes2]


                if p >= lower_bound :
                    ineq = ">="
                    end = ""
                else:
                    ineq = "<"
                    end = "!!!"

                print( "P["+hashC+"] "+ineq+" P["+hash1+"] * P["+hash2+"] "+end )

Now let's actually calculate these values for the method we have chosen to set up and hash the variables.

In [62]:
P = calculate_P()
print(P)

{'HH': 0.6337890625, 'HV': 0.3515625, 'VH': 1.0, 'VV': 0.7744140625}


In [80]:
bell_test( P )

P[HH] >= P[HH] * P[HH] 
P[HV] >= P[HH] * P[HV] 
P[HH] >= P[HV] * P[VH] 
P[HV] >= P[HV] * P[VV] 
P[VH] >= P[VH] * P[HH] 
P[VV] >= P[VH] * P[HV] 
P[VH] >= P[VV] * P[VH] 
P[VV] >= P[VV] * P[VV] 


### Bell test for quantum variables

Let's start by importing everything needed for the quantum program, and setting up the API.

In [68]:
import sys
sys.path.append("../") # go to parent dir
import Qconfig
from IBMQuantumExperience import IBMQuantumExperience
api = IBMQuantumExperience(Qconfig.APItoken, {'url':Qconfig.config["url"]})
from qiskit import QuantumProgram
from qiskit.backends import discover_remote_backends
remote_backends = discover_remote_backends(api)

The initialization of the program itself, including defining the number of qubits and bits it has, is done in the following function.

In [69]:
def initialize_program ():
    
    qp = QuantumProgram()
    qubit = qp.create_quantum_register('qr', 5)
    bit = qp.create_classical_register('cr', 5)
    program = qp.create_circuit( 'bell_test', [qubit], [bit] )
    
    A = qubit[0]
    B = qubit[1]
    a = bit[0]
    b = bit[1]

    return qp, A, B, a, b, program

Now it is time to write your program! Go to the tutorial and pick the *Bell test* mode. This will allow you to experiment in making a quantum program that will prove the unique properties of quantum mechanics. Take the lines of QISKit code that you create and paste them into the empty function below.

In [70]:
def setup_variables ( program, A, B ):
    
    pass # delete this line, and put your program here instead

    program.x(A)

In [71]:
def hash2bit  ( program, qubit, bit, hash ):
    
    if hash=='V':
        program.h( qubit )
        
    program.measure( qubit, bit )
        

In [76]:
def get_results ( qp ):
    
    # submit job
    qp.execute(['bell_test'], backend='ibmqx_qasm_simulator', shots = shots, max_credits = 5, timeout=1)
    print("Job submitted")

    # get job id
    job_id = api.get_jobs(limit=50)[0]['id']

    import time
    data_needed = True
    while data_needed:
        jobs = api.get_jobs(limit=10)
        for job in jobs:
            if job['id']==job_id:
                if job['qasms'][0]['status']=="DONE":
                    stats = job['qasms'][0]['result']['data']['counts']
                    data_needed = False
                    print("Job done")
                else:
                    print("No results yet! We'll wait a while and check again.")
                    time.sleep(600)
    
    p = {}
    for string in stats_raw.keys():
        p[string[4]+string[3]] = stats[string]/shots
                    
    return stats

In [81]:
def calculate_P ():
    
    P = {}
    for hashes in ['HH','HV','VH','VV']:

        for shot in range(shots):
            
            qp, A, B, a, b, program = initialize_program ()

            setup_variables( program, A, B )

            hash2bit ( program, A, a, hashes[0] )
            hash2bit ( program, B, b, hashes[1] )
            
        p = get_results( qp )

        P[hashes] = 0
        for a,b in stats.keys():
            if a==b:
                P[hashes] += p[a+b]

    return P

In [82]:
print( calculate_P() )
print(P)

Job submitted
Job done
Job submitted
Job done
Job submitted
Job done
Job submitted
Job done
{'HH': 0, 'HV': 532, 'VH': 512, 'VV': 522}
{'HH': 0.6337890625, 'HV': 0.3515625, 'VH': 1.0, 'VV': 0.7744140625}


In [ ]:
bell_test( P )